## Importing 'DDW-C18-0000.XLSX'(Language Data)  and 'DDW_PCA0000_2011_Indiastatedist.xlsx'(Census Data) file using pandas

In [1]:
import pandas as pd
data = pd.read_excel('INPUT/DDW-C18-0000.XLSX')
census_data = pd.read_excel('INPUT/DDW_PCA0000_2011_Indiastatedist.xlsx')

### Getting the index of rows containing state data for getting the total population of each state from census data

In [2]:
index = []
for i in range(len(census_data)):
    if census_data['Level'][i] == 'STATE':
        index.append(i)
for i in range(0,len(index),3):
    index[int(i/3)] = index[i]
index = index[:35]
index.insert(0,0)
index

[0,
 3,
 72,
 111,
 174,
 180,
 222,
 288,
 318,
 420,
 636,
 753,
 768,
 819,
 855,
 885,
 912,
 927,
 951,
 1035,
 1095,
 1170,
 1263,
 1320,
 1473,
 1554,
 1563,
 1569,
 1677,
 1749,
 1842,
 1851,
 1857,
 1902,
 2001,
 2016]

### Getting the names of states from census data and language data and checking if they both have the same states with same sequence

In [3]:
state_census = [0]*(len(index))
for i in range(len(index)):
    state_census[i] = census_data['Name'][index[i]]
state_census[0] = 'INDIA'
state_lang = data['Unnamed: 2'][6:].unique()
yes = 0
for i in range(len(state_census)):
    if state_census[i] == state_lang[i]:
        yes = yes+1
print(yes)

36


### Since the states list from both census and language data are same we can directly corresponding the states list of both data
### Getting the total Urban and Rural population for each state from census data

In [4]:
state_pop_urban = [0]*len(index)
state_pop_rural = [0]*len(index)
for i in range(len(index)):
    state_pop_urban[i] = census_data['TOT_P'][index[i]+2]
    state_pop_rural[i] = census_data['TOT_P'][index[i]+1]

### For Urban and Rural population seperately:
### Getting the 'Number speaking second language' and 'Number speaking third language' from language data
### People speaking 3 or more languages is given directly in the data

In [5]:
state_2lang_urban = [0] * len(index)
state_2lang_rural = [0] * len(index)
state_threelang_urban = [0] * len(index)
state_threelang_rural = [0] * len(index)
for i in range(len(index)):
    state_2lang_urban[i] = data['Unnamed: 5'][list(data['Unnamed: 2']).index(state_census[i]) + 20]
    state_2lang_rural[i] = data['Unnamed: 5'][list(data['Unnamed: 2']).index(state_census[i]) + 10]
    state_threelang_urban[i] = data['Unnamed: 8'][list(data['Unnamed: 2']).index(state_census[i]) + 20]
    state_threelang_rural[i] = data['Unnamed: 8'][list(data['Unnamed: 2']).index(state_census[i]) + 10]

### For Urban and Rural Population seperately:
### Getting the counts of people speaking only one language, exactly two languages and three languages or more for each state.
### People speaking only 1 language = Total Population - People speaking 2 or more language
### People speaking exactly 2 languages = People speaking 2 or more language - People speaking 3 or more language

In [6]:
state_onelang_urban = [0] * len(index)
state_onelang_rural = [0] * len(index)
state_twolang_urban = [0] * len(index)
state_twolang_rural = [0] * len(index)
for i in range(len(index)):
    state_onelang_urban[i] = state_pop_urban[i] - state_2lang_urban[i]
    state_onelang_rural[i] = state_pop_rural[i] - state_2lang_rural[i]
    state_twolang_urban[i] = state_2lang_urban[i] - state_threelang_urban[i]
    state_twolang_rural[i] = state_2lang_rural[i] - state_threelang_rural[i]

### Getting the percentage of Urban and Rural population speaking only one language, exactly two languages and three languages

In [7]:
state_onelang_percent_urban = [0]*len(index)
state_onelang_percent_rural = [0]*len(index)
state_twolang_percent_urban = [0]*len(index)
state_twolang_percent_rural = [0]*len(index)
state_threelang_percent_urban = [0]*len(index)
state_threelang_percent_rural = [0]*len(index)
for i in range(len(index)):
    state_onelang_percent_urban[i] = (state_onelang_urban[i] * 100)/state_pop_urban[i]
    state_onelang_percent_rural[i] = (state_onelang_rural[i] * 100)/state_pop_rural[i]
    state_twolang_percent_urban[i] = (state_twolang_urban[i] * 100)/state_pop_urban[i]
    state_twolang_percent_rural[i] = (state_twolang_rural[i] * 100)/state_pop_rural[i]
    state_threelang_percent_urban[i] = (state_threelang_urban[i] * 100)/state_pop_urban[i]
    state_threelang_percent_rural[i] = (state_threelang_rural[i] * 100)/state_pop_rural[i]

### Getting the codes of the states

In [ ]:
state_code = [0] * len(index)
for i in range(len(index)):
    state_code[i] = data['C-18 POPULATION BY BILINGUALISM, TRILINGUALISM, AGE AND SEX'][list(data['Unnamed: 2']).index(state_census[i])]

### Importing ttest_1samp from scipy.stats module to do statistical test and get P-VALUE

In [ ]:
from scipy.stats import ttest_1samp

### Getting P-VALUE

In [ ]:
ratio_pop = [0]*len(state_pop_urban)
ratio_1 = [0]*len(state_pop_urban)
ratio_2 = [0]*len(state_pop_urban)
ratio_3 = [0]*len(state_pop_urban)
p_val = [0]*len(state_pop_urban)
for i in range(len(state_pop_urban)):
    ratio_pop[i] = state_pop_urban[i] / state_pop_rural[i]
    ratio_1[i] = state_onelang_urban[i] / state_onelang_rural[i]
    ratio_2[i] = state_twolang_urban[i] / state_twolang_rural[i]
    ratio_3[i] = state_threelang_urban[i] / state_threelang_rural[i]
    p_val[i] = ttest_1samp([ratio_1[i],ratio_2[i],ratio_3[i]], ratio_pop[i], axis = 0)[1]

### Exporting state codes, male percentage, female percentage and p value to dataframes for one language, two languages and three languages

In [ ]:
data_final = {'state-code':state_code,'urban-percentage':state_onelang_percent_urban,'rural-percentage':state_onelang_percent_rural,'p-value':p_val}
geography_india_a = pd.DataFrame(data_final)
data_final = {'state-code':state_code,'urban-percentage':state_twolang_percent_urban,'rural-percentage':state_twolang_percent_rural,'p-value':p_val}
geography_india_b = pd.DataFrame(data_final)
data_final = {'state-code':state_code,'urban-percentage':state_threelang_percent_urban,'rural-percentage':state_threelang_percent_rural,'p-value':p_val}
geography_india_c = pd.DataFrame(data_final)

### Exporting dataframes to 'geography-india-{part}.csv'

In [ ]:
geography_india_a.to_csv('OUTPUT/geography-india-a.csv',index=False)
geography_india_b.to_csv('OUTPUT/geography-india-b.csv',index=False)
geography_india_c.to_csv('OUTPUT/geography-india-c.csv',index=False)